In [14]:
import torch
import numpy as np
import cv2
from ultralytics import YOLO
import math
from sort import *

In [49]:
cap = cv2.VideoCapture(r"C:\Users\aashutosh kumar\Downloads\People.mp4")
model = YOLO("yolov8l.pt")

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

mask = cv2.imread(r"C:\Projects\People-Counter\Peo_Mask.png")
mask = cv2.resize(mask, (700, 600))
tracker = Sort(max_age = 20,min_hits = 2,iou_threshold = 0.3)
limits = [536,337,606,510]
total_counts = []

while(True):
    r, frame = cap.read()

    if r == True:
        frame = cv2.resize(frame, (700, 600))
        frame_rgn = cv2.bitwise_and(frame, mask) 
        
        detections = np.empty((0,5))
        results = model(frame_rgn, stream=True)
        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                class_id = int(box.cls[0])
                confidence = math.ceil((box.conf[0] * 100)) / 100
                class_Name = classNames[class_id]

                if class_Name == "person" and confidence > 0.4:
                    # cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0),2,4)  
                    curr_array = np.array([x1, y1, x2, y2, confidence])
                    detections = np.vstack((detections, curr_array))
        
        tracking_res = tracker.update(detections)
        
        cv2.line(frame, (limits[0], limits[1]), (limits[2], limits[3]), (255,0,0),4)
        for res in tracking_res:
            x1, x2, y1, y2, id = res
            x1, x2, y1, y2 = int(x1), int(y1), int(x2), int(y2)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (150, 200, 0), 3, 1)

            w = x2 - x1
            h = y2 - y1
            cx = x1 + w // 2
            cy = y1 + h // 2
            
            cv2.circle(frame, (cx,cy), 2, (0,0,255),3)
            # cv2.line(frame, (limits[0], limits[1]), (limits[2], limits[3]), (255,0,0))
            if limits[0] < cx < limits[2] and limits[1] - 90 < cy < limits[1] + 90:
                if id not in total_counts:
                    total_counts.append(id)
                    cv2.line(frame, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5)
            
            cv2.putText(frame,str(len(total_counts)),(25,70),cv2.FONT_HERSHEY_PLAIN,5,(15,241,249),8)


        cv2.imshow("LIVE", frame)
        # cv2.imshow("Mask", frame_rgn)
        if cv2.waitKey(25) & 0xff == ord("p"):
            break

cap.release()
cv2.destroyAllWindows()



0: 576x640 16 persons, 1 train, 1 backpack, 3 handbags, 48.9ms
Speed: 3.9ms preprocess, 48.9ms inference, 3.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 16 persons, 1 train, 1 backpack, 2 handbags, 38.0ms
Speed: 4.0ms preprocess, 38.0ms inference, 3.1ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 17 persons, 1 train, 1 backpack, 2 handbags, 1 cell phone, 38.6ms
Speed: 4.1ms preprocess, 38.6ms inference, 3.5ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 18 persons, 1 train, 1 backpack, 2 handbags, 1 cell phone, 38.7ms
Speed: 4.0ms preprocess, 38.7ms inference, 4.1ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 17 persons, 1 train, 1 backpack, 2 handbags, 1 cell phone, 38.5ms
Speed: 4.0ms preprocess, 38.5ms inference, 4.6ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 16 persons, 1 train, 1 backpack, 1 handbag, 1 cell phone, 39.2ms
Speed: 5.1ms preprocess, 39.2ms inference, 3.0ms postprocess per image at sh